# Association rule mining

In this notebook, you'll implement the basic pairwise association rule mining algorithm.

To keep the implementation simple, you will apply your implementation to a simplified dataset, namely, letters ("items") in words ("receipts" or "baskets"). Having finished that code, you will then apply that code to some grocery store market basket data. If you write the code well, it will not be difficult to reuse building blocks from the letter case in the basket data case.



In [1]:
### Global imports
import dill
from cse6040_devkit import plugins, utils
from cse6040_devkit.training_wheels import run_with_timeout, suppress_stdout
import tracemalloc
from time import time
import re 
import pandas as pd
from copy import deepcopy
from collections import defaultdict
from itertools import combinations
from textwrap import dedent
from operator import itemgetter

utils.add_from_file('update_dd_checker', plugins)
utils.add_from_file('gen_rule_str', utils)
utils.add_from_file('print_rules', utils)


cse6040_devkit.plugins
cse6040_devkit.utils
cse6040_devkit.utils


## Problem definition

Let's say you have a fragment of text in some language. You wish to know whether there are association rules among the letters that appear in a word. In this problem:

- Words are "receipts"
- Letters within a word are "items"

You want to know whether there are _association rules_ of the form, $a \implies b$, where $a$ and $b$ are letters. You will write code to do that by calculating for each rule its _confidence_, $\mathrm{conf}(a \implies b)$. "Confidence" will be another name for an estimate of the conditional probability of $b$ given $a$, or $\mathrm{Pr}[b \,|\, a]$.

## Sample text input

Let's carry out this analysis on a "dummy" text fragment, which graphic designers refer to as the [_lorem ipsum_](https://en.wikipedia.org/wiki/Lorem_ipsum):

In [2]:
### Run Me!!!
latin_text = utils.load_object_from_publicdata('latin_text')


### Exercise 0: (0 points)
**investigate_latin_text**  

**Example:** we have defined `investigate_latin_text` as follows:


This is an ungraded exercise. There is no code to write.
Instead, read the provided Latin text (Lorem Ipsum) and research its meaning.

This function does not return any value.


In [3]:
### Solution - Exercise 0  
def investigate_latin_text():
    print(f'{type(latin_text)=}')
    print(f'Length of latin_text: {len(latin_text)} characters')
    print()
    print(f'First 100 characters of latin_text:\n{latin_text[:100]}')

### Demo function call
investigate_latin_text()

type(latin_text)=<class 'str'>
Length of latin_text: 1744 characters

First 100 characters of latin_text:

Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, 


 


 ---
 <!-- Test Cell Boilerplate -->  
 The test cell below will always pass. Please submit to collect your free points for investigate_latin_text (exercise 0).
 

In [4]:
### Test Cell - Exercise 0  


print('Passed! Please submit.')

Passed! Please submit.


## Data cleaning

Like most data in the real world, this dataset is noisy. It has both uppercase and lowercase letters, words have repeated letters, and there are all sorts of non-alphabetic characters. For our analysis, we should keep all the letters and spaces (so we can identify distinct words), but we should ignore case and ignore repetition within a word.

For example, the eighth word of this text is "error." As an _itemset_, it consists of the three unique letters, $\{e, o, r\}$. That is, treat the word as a set, meaning you only keep the unique letters.

This itemset has three possible _itempairs_: $\{e, o\}$, $\{e, r\}$, and $\{o, r\}$.

> Since sets are unordered, note that we would regard $\{e, o\} = \{o, e\}$, which is why we say there are only three itempairs, rather than six.

Start by writing some code to help "clean up" the input.

### Exercise 1: (2 points)
**normalize_string**  

**Your task:** define `normalize_string` as follows:


Normalize the input string by lowercasing and removing non-alphabetic, non-whitespace characters.

Args:

- s (str): The input string to normalize.

Returns:

- str: A new string containing only lowercase alphabetic characters and whitespace characters, preserving the original order and spacing of retained characters.


> _Clarification_. Scanning the sample text, `latin_text`, you may see things that look like special cases. For instance, `inci[di]dunt` and `[do]`. For these, simply remove the non-alphabetic characters and only separate the words if there is explicit whitespace.
>
> For instance, `inci[di]dunt` would become `incididunt` (as a single word) and `[do]` would become `do` as a standalone word because the original string has whitespace on either side. A period or comma without whitespace would, similarly, just be treated as a non-alphabetic character inside a word _unless_ there is explicit whitespace. So `e pluribus.unum basium` would become `e pluribusunum basium` even though your common-sense understanding might separate `pluribus` and `unum`.
>
> _Hint_. Regard as a whitespace character anything "whitespace-like." That is, consider not just regular spaces, but also tabs, newlines, and perhaps others. To detect whitespaces easily, look for a "high-level" function that can help you do so rather than checking for literal space characters.

In [5]:
### Solution - Exercise 1  
def normalize_string(s):
        
    assert type (s) is str
    s = s.lower()

    return re.sub(r'[^a-z\s]','', s)
        

### Demo function call
demo_s_ex1 = latin_text[:100]
print(f'demo_s_ex1: {demo_s_ex1}\n')
print(f'normalize_string(demo_s_ex1): {normalize_string(demo_s_ex1)}')

demo_s_ex1: 
Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, 

normalize_string(demo_s_ex1): 
sed ut perspiciatis unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium 


 

**The demo should display this printed output.**
```
demo_s_ex1: 
Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, 

normalize_string(demo_s_ex1): 
sed ut perspiciatis unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for normalize_string (exercise 1). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [6]:
### Test Cell - Exercise 1  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=normalize_string,
              ex_name='normalize_string',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to normalize_string did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.14 seconds
memory after test: 3.07 MB
memory peak during test: 4.21 MB
Passed! Please submit.


### Exercise 2: (1 points)
**get_normalized_words**  

**Your task:** define `get_normalized_words` as follows:


Normalize the input string and return a list of its words.

Args:

- s (str): The input string to process.

Returns:
- list of str: A list of words obtained from the normalized string.

Notes:

- The "normalization" process involves lowercasing the string and removing non-alphabetic, non-whitespace characters. (You may find it helpful to call the `normalize_string` function defined earlier.)


In [7]:
### Solution - Exercise 2  
def get_normalized_words(s):

    assert type(s) is str
    s = s.lower()
    
    return re.sub(r'[^a-z\s]', '', s).split()
   

    

### Demo function call
demo_s_ex2 = latin_text[:33]
print(f'demo_s_ex2: {demo_s_ex2}\n')
print(f'{get_normalized_words(demo_s_ex2)=}')

demo_s_ex2: 
Sed ut perspiciatis, unde omnis 

get_normalized_words(demo_s_ex2)=['sed', 'ut', 'perspiciatis', 'unde', 'omnis']


 

**The demo should display this printed output.**
```
demo_s_ex2: 
Sed ut perspiciatis, unde omnis 

get_normalized_words(demo_s_ex2)=['sed', 'ut', 'perspiciatis', 'unde', 'omnis']
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for get_normalized_words (exercise 2). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [8]:
### Test Cell - Exercise 2  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=get_normalized_words,
              ex_name='get_normalized_words',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to get_normalized_words did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.08 seconds
memory after test: 0.05 MB
memory peak during test: 1.41 MB
Passed! Please submit.


### Exercise 3: (2 points)
**make_itemsets_unstructured_text**  

**Your task:** define `make_itemsets_unstructured_text` as follows:


Given unstructured text, return a list of sets corresponding to distinct letters in the normalized words in the text.

Args:

- text (str): The input unstructured text.

Returns:

- list of set: A list of sets, each containing the distinct letters of a normalized word.

Notes:

- The output itemsets should appear in the same order as their corresponding words in the input text.
- The normalization process involves lowercasing the text and removing non-alphabetic, non-whitespace characters. (You may find it helpful to call the `get_normalized_words` function defined earlier.)


In [9]:
def make_itemsets_unstructured_text(text):
    
    #text = text.lower()
    #text_modified = re.sub(r'[^a-z ]', '', text).split()
    ##return text_modified
    words = get_normalized_words(text)
    list_set = [set(word) for word in words]
    return list_set

   
    
### Demo function call
demo_text_ex3 = 'sed \tut, perspiciatis\n und.e omnis'
print(f'demo_text_ex3: {demo_text_ex3}\n')
print(f'{make_itemsets_unstructured_text(demo_text_ex3)=}')

demo_text_ex3: sed 	ut, perspiciatis
 und.e omnis

make_itemsets_unstructured_text(demo_text_ex3)=[{'s', 'e', 'd'}, {'u', 't'}, {'p', 'r', 'c', 'a', 's', 'e', 'i', 't'}, {'u', 'n', 'e', 'd'}, {'n', 'o', 's', 'i', 'm'}]


 

**The demo should display this printed output.**
```
demo_text_ex3: sed 	ut, perspiciatis
 und.e omnis

make_itemsets_unstructured_text(demo_text_ex3)=[{'d', 's', 'e'}, {'t', 'u'}, {'r', 's', 't', 'a', 'c', 'p', 'i', 'e'}, {'d', 'n', 'e', 'u'}, {'s', 'o', 'n', 'i', 'm'}]
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for make_itemsets_unstructured_text (exercise 3). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [10]:
### Test Cell - Exercise 3  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=make_itemsets_unstructured_text,
              ex_name='make_itemsets_unstructured_text',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to make_itemsets_unstructured_text did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.07 seconds
memory after test: 0.07 MB
memory peak during test: 1.97 MB
Passed! Please submit.


## Implementing the basic algorithm

Recall the pseudocode for the algorithm that Rachel and Rich derived together:

![FindAssocRules (pseudocode)](https://ndownloader.figshare.com/files/7635700?private_link=3c473609741895a5cc2c)

In the following series of exercises, let's implement this method. We'll build it "bottom-up," first defining small pieces and working our way toward the complete algorithm. This method allows us to test each piece before combining them.

Observe that the bulk of the work in this procedure is just updating these tables, $T$ and $C$. So your biggest implementation decision is how to store those. A good choice is to use a dictionary.

#### Aside: Default dictionaries

Recall that the overall algorithm requires maintaining a table of item-pair (tuples) counts. It would be convenient to use a dictionary to store this table, where keys refer to item-pairs and the values are the counts.

However, with Python's built-in dictionaries, you always to have to check whether a key exists before updating it. For example, consider this code fragment:

```python
D = {'existing-key': 5} # Dictionary with one key-value pair

D['existing-key'] += 1 # == 6
D['new-key'] += 1  # Error: 'new-key' does not exist!
```

The second attempt causes an error because `'new-key'` is not yet a member of the dictionary. So, a more correct approach would be to do the following:

```python
D = {'existing-key': 5} # Dictionary with one key-value pair

if 'existing-key' not in D:
    D['existing-key'] = 0
D['existing-key'] += 1
   
if 'new-key' not in D:
    D['new-key'] = 0
D['new-key'] += 1
```

This pattern is so common that there is a special form of dictionary, called a _default dictionary_, which is available from the `collections` module: [`collections.defaultdict`](https://docs.python.org/3/library/collections.html?highlight=defaultdict#collections.defaultdict).

When you create a default dictionary, you need to provide a "factory" function that the dictionary can use to create an initial value when the key does *not* exist. For instance, in the preceding example, when the key was not present the code creates a new key with the initial value of an integer zero (0). Indeed, this default value is the one you get when you call `int()` with no arguments:

In [11]:
print(int())

0


In [12]:
from collections import defaultdict

D2 = defaultdict(int) # Empty dictionary

D2['existing-key'] = 5 # Create one key-value pair

D2['existing-key'] += 1 # Update
D2['new-key'] += 1

print(D2)

defaultdict(<class 'int'>, {'existing-key': 6, 'new-key': 1})


### Application in our algorithm: pair_counts

We can use a default dictionary to represent the tables $T$ and $C$ from our algorithm. Let's start with $T$...

_Let's say we have two itemsets:_

```python
itemsets = [set('go'), set('dog')]
```

_Let's set up the default dictionary_

```python
pair_counts = defaultdict(int)
# pair_counts[(a, b)] = count of itemsets which contain both a and b.
```

_Now let's iterate through itemsets and update our table:_

```python
for itemset in itemsets:
    update_pair_counts(pair_counts, itemset)
    print(pair_counts)
```

_We would expect the first print to show a defaultdict containing:_

```python
{('g', 'o'): 1, ('o', 'g'): 1}
```

_We started with an empty `pair_counts`, and updated it with the distinct pairs of letters in the word "go"._

_The second print should show the defaultdict now contains:_

```python
{('g', 'o'): 2, ('o', 'g'): 2, ('d', 'o'): 1, ('o', 'd'): 1, ('d', 'g'): 1, ('g', 'd'): 1}
```

_The pairs ('g', 'o') and ('o', 'g') are in 'dog', so those counts are incremented. The other pairs containing 'd' now have counts of 1._

**_Now we need to implement `update_pair_counts`!_**

### Exercise 4: (2 points)
**update_pair_counts**  

**Your task:** define `update_pair_counts` as follows:


Update the pair_counts default dictionary for all item pairs in the given itemset.

Args:

- pair_counts (defaultdict): The dictionary to update.
- itemset (set): The set of items to consider.

Returns:

- None: The function updates the pair_counts dictionary in place.

Notes:

- You may assume all items in the given itemset are distinct.
- You may also assume the pair_counts dictionary is a default dictionary.


**Hint**

We have imported `combinations` from `itertools` (at the top of the notebook), which is quite useful for iterating over the possible pairs of items within an itemset. This is not the only way to solve the exercise, but we think it's the most straight-forward. 

Feel free to explore the `itertools` module or experiment on your own if you're interested in alternative ways of solving this part of the exercises.

In [13]:
### Solution - Exercise 4  
def update_pair_counts (pair_counts, itemset):
    assert type (pair_counts) is defaultdict
    for (a,b) in combinations(itemset, 2):
            pair_counts[a,b] += 1
            pair_counts[b,a] += 1
    return pair_counts
       
        
   
        
           
   
### Demo function call
demo_pair_counts_ex4 = defaultdict(int)
update_pair_counts(demo_pair_counts_ex4, set('go'))
print(f'After first_itemset: \n{dict(demo_pair_counts_ex4)}')
update_pair_counts(demo_pair_counts_ex4, set('dog'))
print(f'After second_itemset: \n{dict(demo_pair_counts_ex4)}')

After first_itemset: 
{('o', 'g'): 1, ('g', 'o'): 1}
After second_itemset: 
{('o', 'g'): 2, ('g', 'o'): 2, ('o', 'd'): 1, ('d', 'o'): 1, ('g', 'd'): 1, ('d', 'g'): 1}


 

**The demo should display this printed output.**
```
After first_itemset: 
{('g', 'o'): 1, ('o', 'g'): 1}
After second_itemset: 
{('g', 'o'): 2, ('o', 'g'): 2, ('d', 'g'): 1, ('g', 'd'): 1, ('d', 'o'): 1, ('o', 'd'): 1}
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for update_pair_counts (exercise 4). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


Note:

The test uses items which are multi-character strings rather than individual letters. If your solution works for the longer strings, it will also work for the letters. However, the reverse may not be true. 

In [14]:
### Test Cell - Exercise 4  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=plugins.update_dd_checker(update_pair_counts),
              ex_name='update_pair_counts',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to update_pair_counts did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.74 seconds
memory after test: 0.19 MB
memory peak during test: 2.34 MB
Passed! Please submit.


### Application in our algorithm: item_counts

Same idea as above. The plan is to use a default dictionary to model the $C$ table.

```python
# initialize itemsets
itemsets = [set('go'), set('dog')]

# Create item_counts
# item_counts[some_item] = number of itemsets containing the item `some_item` (whatever that may be)
item_counts = defaultdict(int)

# iterate and update
for itemset in itemsets:
    update_item_counts(item_counts, itemset)
    print(item_counts)
```

We expect the prints to be:

```
{'g': 1, 'o': 1}
{'g': 2, 'o': 2, 'd': 1}
```

### Exercise 5: (2 points)
**update_item_counts**  

**Your task:** define `update_item_counts` as follows:


Update the item_counts default dictionary for all items in the given itemset.

Args:

- item_counts (defaultdict): The dictionary to update.
- itemset (set): The set of items to consider.

Returns:

- None: The function updates the item_counts dictionary in place.

Notes:

- You may assume all items in the given itemset are distinct.
- You may also assume the item_counts dictionary is a default dictionary.


In [15]:
### Solution - Exercise 5  
def update_item_counts(item_counts, itemset):
    assert type(item_counts) is defaultdict
    
    for i in itemset:
        item_counts[i] +=1
    
    return item_counts
        

    

### Demo function call
demo_item_counts_ex5 = defaultdict(int)
update_item_counts(demo_item_counts_ex5, set('go'))
print(f'After first_itemset: \n{dict(demo_item_counts_ex5)}')
update_item_counts(demo_item_counts_ex5, set('dog'))
print(f'After second_itemset: \n{dict(demo_item_counts_ex5)}')

After first_itemset: 
{'o': 1, 'g': 1}
After second_itemset: 
{'o': 2, 'g': 2, 'd': 1}


 

**The demo should display this printed output.**
```
After first_itemset: 
{'g': 1, 'o': 1}
After second_itemset: 
{'g': 2, 'o': 2, 'd': 1}
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for update_item_counts (exercise 5). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [16]:
### Test Cell - Exercise 5  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=plugins.update_dd_checker(update_item_counts),
              ex_name='update_item_counts',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to update_item_counts did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.09 seconds
memory after test: 0.03 MB
memory peak during test: 1.39 MB
Passed! Please submit.


### Application in our algorithm: Confidence calculation

After populating `pair_counts` and `item_counts` from a collection of "receipts", we have the data in a form where it can be analyzed. It's time to implement the pairwise association rules by calculating the confidence!

Remember, $\mathrm{conf}(a \Rightarrow b)$ is $\text{Pr}(b|a)$ - the probability that item $b$ is in a basket given that item $a$ is in the basket.

We can estimate this with our "counts" dictionaries.

`conf[(a, b)] = pair_counts[(a, b)] / item_counts[a]`

If we choose `a` and `b` by iterating over the keys of `pair_counts`, we will have a confidence calculation for each pair of items.

### Aside

Printing the confidence rules using the built-in `print` leads to some messy output. We have provided `utils.print_rules` to print them nicely in the demos for the exercises below. Remember that the rules being printed are still dictionaries.

### Exercise 6: (1 points)
**create_rules_from_counts**  

**Your task:** define `create_rules_from_counts` as follows:


Create association rules from pair counts and item counts.

Args:

- pair_counts (dict): A dictionary mapping pairs (a, b) to their co-occurrence counts.
- item_counts (dict): A dictionary mapping items to their individual occurrence counts.

Returns:
- dict: A dictionary mapping pairs (a, b) to their confidence values. In other words the pair (a,b) maps to $\mathrm{conf}(a \Rightarrow b)$.

Note:
- You may assume that for every (a, b) in pair_counts, a is present in item_counts with a non-zero count.


In [17]:
### Solution - Exercise 6  
def create_rules_from_counts(pair_counts, item_counts):
    rules = {} # (item_a, item_b) -> conf (item_a => item_b)
    
    for (a, b) in pair_counts:
        
        conf = pair_counts[(a, b)] / item_counts[a]
        rules[(a,b)] = conf
    return rules

### Demo function call
demo_item_counts_ex7 = {'blue fish': 44, 
                    'one fish': 47, 
                    'red fish': 100, 
                    'two fish': 74}
demo_pair_counts_ex7 = {('blue fish', 'one fish'): 16,
                    ('one fish', 'blue fish'): 16,
                    ('blue fish', 'red fish'): 36,
                    ('red fish', 'blue fish'): 36,
                    ('blue fish', 'two fish'): 27,
                    ('two fish', 'blue fish'): 27,
                    ('one fish', 'red fish'): 38,
                    ('red fish', 'one fish'): 38,
                    ('one fish', 'two fish'): 28,
                    ('two fish', 'one fish'): 28,
                    ('red fish', 'two fish'): 59,
                    ('two fish', 'red fish'): 59}
demo_rules = create_rules_from_counts(demo_pair_counts_ex7, demo_item_counts_ex7)
utils.print_rules(demo_rules)

conf(blue fish => red fish) = 0.818
conf(one fish => red fish) = 0.809
conf(two fish => red fish) = 0.797
conf(blue fish => two fish) = 0.614
conf(one fish => two fish) = 0.596
conf(red fish => two fish) = 0.590
conf(red fish => one fish) = 0.380
conf(two fish => one fish) = 0.378
conf(two fish => blue fish) = 0.365
conf(blue fish => one fish) = 0.364
conf(red fish => blue fish) = 0.360
conf(one fish => blue fish) = 0.340


 

**The demo should display this printed output.**
```
conf(blue fish => red fish) = 0.818
conf(one fish => red fish) = 0.809
conf(two fish => red fish) = 0.797
conf(blue fish => two fish) = 0.614
conf(one fish => two fish) = 0.596
conf(red fish => two fish) = 0.590
conf(red fish => one fish) = 0.380
conf(two fish => one fish) = 0.378
conf(two fish => blue fish) = 0.365
conf(blue fish => one fish) = 0.364
conf(red fish => blue fish) = 0.360
conf(one fish => blue fish) = 0.340
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for create_rules_from_counts (exercise 6). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [18]:
### Test Cell - Exercise 6  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=create_rules_from_counts,
              ex_name='create_rules_from_counts',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to create_rules_from_counts did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.09 seconds
memory after test: 0.04 MB
memory peak during test: 1.39 MB
Passed! Please submit.


### Application in our algorithm: confidence threshold

The last step in our algorithm is to filter out rules with a low confidence. We want to remove any rules which do not have confidence that is at least some threshold value. The value itself should be parameterized so that users can tune it to their own application and use-case.

To accomplish the task, we can create a new dictionary

```python
filtered_rules = {}
```

_Then iterate over the key/value (`rule`/`conf`) pairs in the unfiltered rules, adding any pair with confidence of at least the threshold to the new dictionary._

```python
filtered_rules.update({rule: conf})
```  
  
_When the iterations are all finished, `filtered_rules` will be a dictionary containing only the rules where the confidence value is at least the threshold value._  


### Exercise 7: (1 points)
**filter_rules_by_conf**  

**Your task:** define `filter_rules_by_conf` as follows:


Filter rules by confidence threshold.

Args:

- rules (dict): A dictionary mapping pairs (a, b) to confidence values.
- threshold (float): The minimum confidence threshold.

Returns:

- dict: A dictionary containing only the rules with confidence >= threshold.


In [19]:
### Solution - Exercise 7  
def filter_rules_by_conf(rules, threshold):
    filtered_rules = {}
    for key, val in rules.items():
        if val >= threshold:
            filtered_rules.update({key:val})
    return filtered_rules
    
### Demo function call
demo_rules_ex7 = {('blue fish', 'one fish'): 0.36363636363636365,
                    ('one fish', 'blue fish'): 0.3404255319148936,
                    ('blue fish', 'red fish'): 0.8181818181818182,
                    ('red fish', 'blue fish'): 0.36,
                    ('blue fish', 'two fish'): 0.6136363636363636,
                    ('two fish', 'blue fish'): 0.36486486486486486,
                    ('one fish', 'red fish'): 0.8085106382978723,
                    ('red fish', 'one fish'): 0.38,
                    ('one fish', 'two fish'): 0.5957446808510638,
                    ('two fish', 'one fish'): 0.3783783783783784,
                    ('red fish', 'two fish'): 0.59,
                    ('two fish', 'red fish'): 0.7972972972972973}
demo_threshold_ex7 = 0.59
demo_result_ex7 =filter_rules_by_conf(demo_rules_ex7, demo_threshold_ex7)
utils.print_rules(demo_result_ex7)

conf(blue fish => red fish) = 0.818
conf(one fish => red fish) = 0.809
conf(two fish => red fish) = 0.797
conf(blue fish => two fish) = 0.614
conf(one fish => two fish) = 0.596
conf(red fish => two fish) = 0.590


 

**The demo should display this printed output.**
```
conf(blue fish => red fish) = 0.818
conf(one fish => red fish) = 0.809
conf(two fish => red fish) = 0.797
conf(blue fish => two fish) = 0.614
conf(one fish => two fish) = 0.596
conf(red fish => two fish) = 0.590
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for filter_rules_by_conf (exercise 7). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [20]:
### Test Cell - Exercise 7  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=filter_rules_by_conf,
              ex_name='filter_rules_by_conf',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to filter_rules_by_conf did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.09 seconds
memory after test: 0.03 MB
memory peak during test: 1.39 MB
Passed! Please submit.


## Polishing the algorithm

So far we have the building blocks to implement the pairwise association mining algorithm.

1. Parse "receipts" into sets of distinct "items".
2. Process the itemsets into co-occurrence and occurrence counts.
3. Process the counts into confidence rules.
4. Filter the confidence rules based on a threshold.

We're off to a good start, but there's still a few things to sort out...

### How to make itemsets a more generic concept

Unfortunately, `make_itemsets_unstructured_text` is very specific to the "words are receipts; letters are items" use-case. It would be nice to have another way to make item sets from different kinds of data. 

Let's look at some real data which someone was nice enouth to prepare for a similar exercise designed for the R programming language. (Earlier versions of this notebook linked to a now defunct blog post with details on the original.)

Run the next two cells to load and explore this data.

In [21]:
### Run Me!!!
groceries_file = utils.load_object_from_publicdata('groceries_file')


The code below displays the first 10 lines from `groceries_file`.

In [22]:
print(f'{type(groceries_file)=}')
print(f'Length of groceries_file: {len(groceries_file)} characters')
print('\nContent Preview:')
print('\n'.join(groceries_file.splitlines()[:10]))

type(groceries_file)=<class 'str'>
Length of groceries_file: 500843 characters

Content Preview:
citrus fruit,semi-finished bread,margarine,ready soups
tropical fruit,yogurt,coffee
whole milk
pip fruit,yogurt,cream cheese ,meat spreads
other vegetables,whole milk,condensed milk,long life bakery product
whole milk,butter,yogurt,rice,abrasive cleaner
rolls/buns
other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer)
pot plants
whole milk,cereals


The data is organized differently than in our unstructured text. It's in a csv-like format.

- Each "receipt" is a line.
- The "items" in each line are separated by commas (',').

To parse this csv string into itemsets, we need to:

- Create an empty list to store the final output.
- Divide the data into individual lines.
- For each line
  - Split by commas.
  - Create a set out of the result.
  - Append the set to the final output.

### Exercise 8: (3 points)
**make_itemsets_csv**  

**Your task:** define `make_itemsets_csv` as follows:


Given a CSV string where each line represents a receipt, return a list of sets corresponding to the distinct items in each receipt.

Args:

- csv_str (str): The input CSV string.
    - Each line corresponds to a receipt.
    - Items in each receipt are separated by commas.

Returns:
- list of sets: A list where each element is a set of distinct items from a receipt.


In [23]:
### Solution - Exercise 8  
def make_itemsets_csv(csv_str):
    list = []
    csv = csv_str.split('\n')
    for line in csv:
        list.append(set(line.split(',')))
    
    
    
    return list

###Create an empty list to store the final output.
#Divide the data into individual lines.
#For each line
#Split by commas.
#Create a set out of the result.
#Append the set to the final output.

### Demo function call
demo_csv_str_ex8 = dedent('''\
                        milk,eggs,peanut butter,oatmeal
                        butter,pancake mix,maple syrup
                        dog treats,milk,milk
                        ''').strip()
print(f'"""{demo_csv_str_ex8}"""')
print(f'{make_itemsets_csv(demo_csv_str_ex8)=}')

"""milk,eggs,peanut butter,oatmeal
butter,pancake mix,maple syrup
dog treats,milk,milk"""
make_itemsets_csv(demo_csv_str_ex8)=[{'peanut butter', 'oatmeal', 'eggs', 'milk'}, {'pancake mix', 'butter', 'maple syrup'}, {'dog treats', 'milk'}]


 

**The demo should display this printed output.**
```
"""milk,eggs,peanut butter,oatmeal
butter,pancake mix,maple syrup
dog treats,milk,milk"""
make_itemsets_csv(demo_csv_str_ex8)=[{'eggs', 'milk', 'peanut butter', 'oatmeal'}, {'maple syrup', 'butter', 'pancake mix'}, {'milk', 'dog treats'}]
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for make_itemsets_csv (exercise 8). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [24]:
### Test Cell - Exercise 8  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=make_itemsets_csv,
              ex_name='make_itemsets_csv',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to make_itemsets_csv did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 0.07 seconds
memory after test: 0.03 MB
memory peak during test: 1.38 MB
Passed! Please submit.


### Aside: Higher order functions

Python functions are _objects_ (like strings, integers, lists, etc.). As objects, functions can be passed as arguments to other functions. You have seen this before when passing the `key` argument to `sorted` in Notebook 1; we passed a function as that argument!

Let's say we need to write some function which pre-processes a string then parses it into a list of words, but we don't know what "pre-process" looks like. We can still do it, but the user is going to have to define the preprocessing logic on their side. Our function will stitch in the logic at runtime.

Here's our function:

In [25]:
def preprocess_and_parse(s, preprocessor):
    """
    Preprocess the string `s` using the function `preprocessor`, then parse the result into a list.
    
    Args:
    - s (str): The input string to preprocess and parse.
    - preprocessor (function): A function that takes a string as input and returns a processed string.
    
    Returns:
    - list: The parsed list from the preprocessed string.
    """
    preprocessed = preprocessor(s)
    return preprocessed.split()

The syntax is simple, just call `preprocessor`. Whenever the `preprocess_and_parse` is called, it's up to the caller to supply a function that can be used the same way `preprocessor` is used. 

The docstring requires "A function that takes a string as input and returns a processed string."

Here's a few examples:

In [26]:
def lowercase_preprocessor(s):
    return s.lower()

def remove_vowels_preprocessor(s):
    return ''.join([c for c in s if c.lower() not in 'aeiou'])

def identity_preprocessor(s):
    return s

def alternating_case_preprocessor(s):
    return ''.join([c.lower() if i % 2 == 0 else c.upper() for i, c in enumerate(s)])

# Demo function calls
demo_string = "I Love Python! It's Awesome."

print(f'{preprocess_and_parse(demo_string, lowercase_preprocessor)=}\n')
print(f'{preprocess_and_parse(demo_string, remove_vowels_preprocessor)=}\n')
print(f'{preprocess_and_parse(demo_string, identity_preprocessor)=}\n')
print(f'{preprocess_and_parse(demo_string, alternating_case_preprocessor)=}\n')

preprocess_and_parse(demo_string, lowercase_preprocessor)=['i', 'love', 'python!', "it's", 'awesome.']

preprocess_and_parse(demo_string, remove_vowels_preprocessor)=['Lv', 'Pythn!', "t's", 'wsm.']

preprocess_and_parse(demo_string, identity_preprocessor)=['I', 'Love', 'Python!', "It's", 'Awesome.']

preprocess_and_parse(demo_string, alternating_case_preprocessor)=['i', 'lOvE', 'PyThOn!', "It's", 'aWeSoMe.']



This is a pretty powerful concept which allows us to write re-usable functions which can take other functions as arguments to adapt to various use-cases.

### Putting it together

It's time to write one function to parse the input data into confidence rules from end to end. These are the high-level requirements.

1. Parse "receipts" into sets of distinct "items".
    - The user should be able to define how this is done.
2. Process the itemsets into co-occurrence and occurrence counts.
3. Process the counts into confidence rules.
4. Filter the confidence rules based on a threshold.
5. Filter the confidence rules based on a minimum occurrence count.

There's two new bits here:

- The _user_ defines how the source data gets processed into itemsets.
    - We will implement this by taking a _function as an argument_.
- There is an additional filter.
    - We will have to make another pass at the rules and cross-reference the occurrence counts to decide which rules to keep under this filter.

### Exercise 9: (4 points)
**create_rules_from_source**  

**Your task:** define `create_rules_from_source` as follows:


Create association rules from a data source using the provided itemset maker function, filtering by confidence threshold and minimum item count.

Args:

- source: The data source (e.g., unstructured text or CSV string).
- itemset_maker (function): A function that takes the source as input and returns a list of itemsets (e.g. make_itemsets_csv, make_itemsets_unstructured_text).
- conf_threshold (float): The minimum confidence threshold for filtering rules.
- min_count (int): The minimum count for the antecedent item in the rules.
    - I.e., `rules[(a, b)]` is only included in the output if `a` appears in at least `min_count` itemsets.

Returns:

- dict: A dictionary mapping pairs (a, b) to their confidence values, filtered by the specified thresholds.


Note:

- The demos below will not work if you have not correctly defined `make_itemsets_unstructured_text` or `make_itemsets_csv`. 
- The test cell uses entirely new functions as `itemset_maker` values.

In [34]:
### Solution - Exercise 9  
def create_rules_from_source(source, itemset_maker, conf_threshold, min_count):
    itemsets = itemset_maker(source)
    pair_counts = defaultdict(int)
    item_counts = defaultdict(int)
    for itemset in  itemsets:
        update_pair_counts(pair_counts, itemset)
        update_item_counts(item_counts, itemset)
    

        rules = {} # (item_a, item_b) -> conf (item_a => item_b)
    
    for (a, b) in pair_counts:
        if item_counts[a] >= min_count:
            conf = pair_counts[(a, b)] / item_counts[a]
            if conf >= conf_threshold:
                rules[(a,b)] = conf
                  
    return rules
    
        
    
### Demo function call
latin_rules = create_rules_from_source(latin_text, make_itemsets_unstructured_text, 0.75, 15)
grocery_rules = create_rules_from_source(groceries_file, make_itemsets_csv, 0.6, 10)
print('Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0')
utils.print_rules(latin_rules)
print()
print('Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10')
utils.print_rules(grocery_rules)

Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0
conf(q => u) = 1.000
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750

Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10
conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600


 

**The demo should display this printed output.**
```
Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0
conf(q => u) = 1.000
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750

Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10
conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for create_rules_from_source (exercise 9). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [35]:
### Test Cell - Exercise 9  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=create_rules_from_source,
              ex_name='create_rules_from_source',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=30)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to create_rules_from_source did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 0.00 MB
Test duration: 1.50 seconds
memory after test: 0.42 MB
memory peak during test: 30.23 MB
Passed! Please submit.


## Application

Our goal was to parse structured data into association rules. These are useful in their own right, offering insights into which items should be offered far apart in a store to encourage customers to traverse the whole floor spare. We can use the rules in different ways.

Say we want to look at association rules for all the stores in a region to predict which associations we can count on holding true for a new store.

To do that we would need a collection of rules, and we simply take the set intersection of all the keys (recall rules are modeled as dictionaries mapping ordered item pairs to confidence values). There is one edge case - what if the list of rules is empty? One option is to return `None`. This distinguishes the "empty rules list" case from the "no common rules" case, and is what we're going to implement. (There are other options, like raising an error, which is covered in a later notebook.)

To handle this edge case, we can use this pattern:

- Initialize variable for collecting the intersection as `None`.
  - i.e. `result = None`.
- Iterate over every "rules" dictionary in the rules_list.
  - If `result is None`, set result to the current `rules` keys.
  - Otherwise, update `result` to be the intersection of `result` and the current `rules` keys.

The cell below loads the English translation of the Lorem Ipsum, which will be used to derive rules for comparison with Latin rules. Let's see what letter pairs are common to both languages!

In [30]:
### Run Me!!!
english_text = utils.load_object_from_publicdata('english_text')


### Exercise 10: (2 points)
**common_rules**  

**Your task:** define `common_rules` as follows:


Given a list of rules dictionaries, return the set of rules (keys) that are common to all dictionaries.

Args:

- rules_list (list of dict): A list where each element is a dictionary mapping pairs (a, b) to confidence values.

Returns:

- set or None: A set of pairs (a, b) that are present as keys in all dictionaries in the input list. If the input list is empty, return None.


In [31]:
### Solution - Exercise 10  
def common_rules(rules_list):
    if not rules_list:
        return None
    common_rules = set(rules_list[0].keys())
    for dict in rules_list[1:]:
        common_rules.intersection_update(dict.keys())
    return common_rules
        
    

### Demo function call
latin_rules = create_rules_from_source(latin_text, make_itemsets_unstructured_text, 0.8, 3)
english_rules = create_rules_from_source(english_text, make_itemsets_unstructured_text, 0.8, 3)
print('Latin rules:')
utils.print_rules(latin_rules)
print('\nEnglish rules:')
utils.print_rules(english_rules)
print('\nCommon rules:')
demo_result_ex10 = common_rules([latin_rules, english_rules])
utils.print_rules(demo_result_ex10)

Latin rules:
conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800

English rules:
conf(x => e) = 1.000
conf(j => e) = 1.000
conf(q => u) = 1.000
conf(q => e) = 1.000

Common rules:
x => e
q => u


 

**The demo should display this printed output.**
```
Latin rules:
conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800

English rules:
conf(x => e) = 1.000
conf(j => e) = 1.000
conf(q => e) = 1.000
conf(q => u) = 1.000

Common rules:
x => e
q => u
```


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for common_rules (exercise 10). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [32]:
### Test Cell - Exercise 10  


from cse6040_devkit.tester_fw.testers import Tester
from yaml import safe_load
from time import time

tracemalloc.start()
mem_start, peak_start = tracemalloc.get_traced_memory()
print(f"initial memory usage: {mem_start/1024/1024:.2f} MB")

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    executor = dill.load(f)

@run_with_timeout(error_threshold=200.0, warning_threshold=100.0)
@suppress_stdout
def execute_tests(**kwargs):
    return executor(**kwargs)


# Execute test
start_time = time()
passed, test_case_vars, e = execute_tests(func=common_rules,
              ex_name='common_rules',
              key=b'uB5AD-6LZ4KH6PExkCGTzp065lKUINubYeq5q9rcV00=', 
              n_iter=32)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars
duration = time() - start_time
print(f"Test duration: {duration:.2f} seconds")
current_memory, peak_memory = tracemalloc.get_traced_memory()
print(f"memory after test: {current_memory/1024/1024:.2f} MB")
print(f"memory peak during test: {peak_memory/1024/1024:.2f} MB")
tracemalloc.stop()
if e: raise e
assert passed, 'The solution to common_rules did not pass the test.'



print('Passed! Please submit.')

initial memory usage: 1.10 MB
Test duration: 0.09 seconds
memory after test: 1.40 MB
memory peak during test: 30.23 MB
Passed! Please submit.
